# Testing `2024-11-17-rev1-000` Model

In [1]:
import sys
import os

colab = 'google.colab' in sys.modules

In [2]:
if colab:
    os.environ['PROJECT_DIR'] = project_dir = '/content/Foggy-CycleGAN'
    replace = True
    if os.path.isdir(project_dir):
        choice = input("Project already exists in folder " +
                       "{}\nDelete the files and pull again? Enter Y/(N):\n"
                       .format(project_dir))
        if choice.lower() == 'y':
            !rm -r $PROJECT_DIR
            print("Deleted folder {}".format(project_dir))
        else:
            replace = False
            print("Nothing was changed.")
    if replace:
        !cd /content && git clone https://github.com/ghaiszaher/Foggy-CycleGAN.git
        print("Project cloned to " + project_dir)
    os.chdir(project_dir)
    print("Done.")

Cloning into 'Foggy-CycleGAN'...
remote: Enumerating objects: 937, done.
remote: Counting objects: 100% (130/130), done.
remote: Compressing objects: 100% (57/57), done.
remote: Total 937 (delta 92), reused 85 (delta 71), pack-reused 807 (from 1)
Receiving objects: 100% (937/937), 68.32 MiB | 35.67 MiB/s, done.
Resolving deltas: 100% (562/562), done.
Project cloned to /content/Foggy-CycleGAN
Done.


In [3]:
from lib.tools import create_dir

output = './weights/generator_clear2fog.weights.h5'
if not os.path.exists(output):
    import gdown
    file_id = '10cBLNuofYGbyrC6DKRycD-fmx0Wh7BCr'
    create_dir('./weights/')
    print("Downloading weights...")
    gdown.download(f'https://drive.google.com/uc?id={file_id}', output, quiet=False)
    print("Weights downloaded.")
else:
    print("Weights already downloaded.")

Downloading...
From (original): https://drive.google.com/uc?id=10cBLNuofYGbyrC6DKRycD-fmx0Wh7BCr
From (redirected): https://drive.google.com/uc?id=10cBLNuofYGbyrC6DKRycD-fmx0Wh7BCr&confirm=t&uuid=acfa52ff-1ef9-49a9-8834-2bb32cc68690
To: /content/Foggy-CycleGAN/weights/generator_clear2fog.weights.h5
100%|██████████| 218M/218M [00:03<00:00, 70.1MB/s]

Weights downloaded.


In [4]:
from lib.models import ModelsBuilder

models_builder = ModelsBuilder()
generator_clear2fog = models_builder.build_generator()
generator_clear2fog.load_weights('./weights/generator_clear2fog.weights.h5')

In [7]:
import os
import tensorflow as tf
import pandas as pd
from lib.plot import plot_clear2fog_intensity
from lib.dataset import DatasetInitializer

# 初始化 DatasetInitializer
datasetInit = DatasetInitializer(1024,1024)

# 參數設置
input_folder = '/content/drive/MyDrive/full_aerial_images'  # 輸入圖片資料夾
output_folder = './custom-images/output/'  # 輸出圖片資料夾
fog_intensity = 0.4  # 霧化強度
output_csv = './custom-images/results.csv'  # SSIM 和 PSNR 結果保存路徑

# 創建輸出資料夾（如果不存在）
os.makedirs(output_folder, exist_ok=True)

# 初始化指標
ssim_scores = []
psnr_scores = []
image_names = []

# 遍歷輸入資料夾中的所有圖片
for image_name in os.listdir(input_folder):
    input_path = os.path.join(input_folder, image_name)
    output_path = os.path.join(output_folder, image_name)

    try:
        # 讀取和預處理圖片
        image_clear = tf.io.decode_png(tf.io.read_file(input_path), channels=3)
        image_clear, _ = datasetInit.preprocess_image_test(image_clear, 0)

        # 生成霧化圖片
        _, foggy_image = plot_clear2fog_intensity(generator_clear2fog, image_clear, fog_intensity)

        # 計算 SSIM 和 PSNR
        ssim = tf.image.ssim(image_clear, foggy_image, max_val=1.0).numpy()
        psnr = tf.image.psnr(image_clear, foggy_image, max_val=1.0).numpy()
        ssim_scores.append(ssim)
        psnr_scores.append(psnr)
        image_names.append(image_name)

        # 保存生成的霧化圖片
        foggy_image_uint8 = tf.image.convert_image_dtype(foggy_image, dtype=tf.uint8)
        encoded_image = tf.io.encode_png(foggy_image_uint8)
        tf.io.write_file(output_path, encoded_image)
        print(f"Processed {image_name}: SSIM={ssim:.4f}, PSNR={psnr:.4f}")

    except Exception as e:
        print(f"Error processing {image_name}: {e}")

# 計算平均 SSIM 和 PSNR
mean_ssim = sum(ssim_scores) / len(ssim_scores) if ssim_scores else 0
mean_psnr = sum(psnr_scores) / len(psnr_scores) if psnr_scores else 0

# 保存 SSIM 和 PSNR 結果到 CSV
results_df = pd.DataFrame({
    "Image Name": image_names,
    "SSIM": ssim_scores,
    "PSNR (dB)": psnr_scores
})
mean_row = pd.DataFrame({
    "Image Name": ["Mean"],
    "SSIM": [mean_ssim],
    "PSNR (dB)": [mean_psnr]
})


Error processing 000000801.jpg: Input 0 of layer "functional_15" is incompatible with the layer: expected shape=(None, 256, 256, 3), found shape=(1, 1024, 1024, 3)
Error processing 000000824.jpg: Input 0 of layer "functional_15" is incompatible with the layer: expected shape=(None, 256, 256, 3), found shape=(1, 1024, 1024, 3)
Error processing 000000812.jpg: Input 0 of layer "functional_15" is incompatible with the layer: expected shape=(None, 256, 256, 3), found shape=(1, 1024, 1024, 3)
Error processing 000000807.jpg: Input 0 of layer "functional_15" is incompatible with the layer: expected shape=(None, 256, 256, 3), found shape=(1, 1024, 1024, 3)
Error processing 000000828.jpg: Input 0 of layer "functional_15" is incompatible with the layer: expected shape=(None, 256, 256, 3), found shape=(1, 1024, 1024, 3)
Error processing 000000816.jpg: Input 0 of layer "functional_15" is incompatible with the layer: expected shape=(None, 256, 256, 3), found shape=(1, 1024, 1024, 3)
Error processing

KeyboardInterrupt: 

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
